<a href="https://colab.research.google.com/github/Jasonhcwong/face-recognition-system/blob/main/benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required libraries and clone the repo

In [1]:
!pip install scikit-learn scipy==1.8.1 mxnet  mtcnn scikit-image

!git clone https://github.com/Jasonhcwong/face-recognition-system.git
%cd face-recognition-system


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'face-recognition-system' already exists and is not an empty directory.
/content/face-recognition-system


# Import required libraries

In [2]:
from verification import evaluate
from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate
import mxnet as mx
import numpy as np
import pickle
import os
import tensorflow as tf

# Load the model and run the benchmark

In [3]:
# load Labeled Faces in the Wild Benchmark file "lfw.bin"
def load_data(db_name, image_size, eval_db_path):
    bins, issame_list = pickle.load(open(os.path.join(eval_db_path, db_name+'.bin'), 'rb'), encoding='bytes')
    datasets = np.empty((len(issame_list)*2, image_size[0], image_size[1], 3))

    for i in range(len(issame_list)*2):
        _bin = bins[i]
        img = mx.image.imdecode(_bin).asnumpy()
        img = img - 127.5
        img = img * 0.0078125
        datasets[i, ...] = img
        i += 1
        if i % 1000 == 0:
            print('loading bin', i)

    return datasets, issame_list

# directory where "lfw.bin" is located
eval_db_path = './'
data_set = load_data('lfw', [112,112], eval_db_path)
data_sets, issame_list = data_set

# load and run Tensorflow saved_model
train_model = tf.keras.models.load_model('./ckpt/epoch_13')
model = tf.keras.Model(train_model.get_layer('input').input, train_model.get_layer('embedding').output, trainable=False)
result = model.predict(data_sets, batch_size=1, verbose=1)

# calculate performance metrics
tpr, fpr, accuracy, val, val_std, far = evaluate(result, issame_list, nrof_folds=10)
print('Accuracy: %1.3f+-%1.3f' % (np.mean(accuracy), np.std(accuracy)))
print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))
print('fpr and tpr: %1.3f %1.3f' % (np.mean(fpr, 0), np.mean(tpr, 0)))

auc = metrics.auc(fpr, tpr)
print('Area Under Curve (AUC): %1.3f' % auc)
eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
print('Equal Error Rate (EER): %1.3f' % eer)

loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
12000/12000 [==============================] - 98s 8ms/step
thresholds max: 1.41 <=> min: 1.3900000000000001
Accuracy: 0.987+-0.006
Validation rate: 0.95733+-0.01788 @ FAR=0.00100
fpr and tpr: 0.502 0.818
Area Under Curve (AUC): 0.999
Equal Error Rate (EER): 0.014
